# Forchheimer equation

In this tutorial we present how to solve a Forchheimer equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
(k^{-1} + \beta\Vert q \Vert) {q} + \nabla p = {0}\\
\nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \qquad p = 1 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [2]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

We create now the grid, since we will use a Raviart-Thomas approximation for ${q}$ we are restricted to simplices. In this example we consider a bi-dimensional structured grid, but the presented code will work also in 1d and 3d. PyGeoN works with mixed-dimensional grids, so we need to convert the grid.

In [3]:
N = 20
sd = pp.CartGrid([N] * 2, [1] * 2)
# convert the grid into a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([sd])

With the following code we set the data, in particular the permeability tensor and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [4]:
keyword = "flow"
bc_val, bc_ess, bc_val_ess, source = [], [], [], []
for sd, data in mdg.subdomains(return_data=True):
    # permeability tensor
    perm = pp.SecondOrderTensor(np.ones(sd.num_cells))
    
    # with the following steps we identify the portions of the boundary
    # to impose the boundary conditions
    b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]

    b_face_centers = sd.face_centers[:, b_faces]
    out_flow = b_face_centers[1, :] == 1 # TOP
    in_flow = b_face_centers[1, :] == 0 # BOTTOM
    no_flow = np.logical_or.reduce((b_face_centers[0, :] == 1, b_face_centers[0, :] == 0))

    faces, _, sign = sps.find(sd.cell_faces)
    sign = sign[np.unique(faces, return_index=True)[1]]
        
    bc_val.append(np.hstack((np.zeros(sd.num_faces), np.zeros(sd.num_cells))))
    bc_val[-1][b_faces[in_flow]] = -sign[b_faces[in_flow]] * np.ones(b_faces[in_flow].size) # pressure boundary

    bc_ess.append(np.hstack((np.zeros(sd.num_faces), np.zeros(sd.num_cells))))
    bc_ess[-1][b_faces[no_flow]] = True   
    #bc_ess[-1][b_faces[out_flow]] = True

    bc_val_ess.append(np.hstack((np.zeros(sd.num_faces), np.zeros(sd.num_cells))))
    #bc_val_ess[-1][b_faces[out_flow]] = sd.face_areas[b_faces[out_flow]] # flux boundary
    
    source.append(np.zeros(sd.num_cells))

    parameters = {
        "second_order_tensor": perm,
    }
    data[pp.PARAMETERS] = {keyword: parameters}
    data[pp.DISCRETIZATION_MATRICES] = {keyword: {}}
    
source = np.hstack(source)
bc_val = np.hstack(bc_val)
bc_ess = np.hstack(bc_ess).astype(bool)

Once the data are assigned to the mixed-dimensional grid, we construct the matrices. In particular, the linear system associated with the equation is given as
$$
\left(
\begin{array}{cc} 
M(q^n) & -B^\top\\
B & 0
\end{array}
\right)
\left(
\begin{array}{c} 
q^{n+1}\\ 
p^{n+1}
\end{array}
\right)
=\left(
\begin{array}{c} 
p_{\partial}\\ 
0
\end{array}
\right)
$$<br>
where $p_{\partial}$ is the vector associated to the pressure boundary contions. To construct the saddle-point problem, we rely on the `scipy.sparse` function `bmat`. Once the matrix is created, we also construct the right-hand side containing the boundary conditions.

In [28]:
cell_mass = pg.cell_mass(mdg)
div = pg.div(mdg)
face_proj = pg.proj_faces_to_cells(mdg, discr=pp.MVEM(keyword))

# get the degrees of freedom for each variable
cell_dof, face_dof = div.shape
dofs = np.cumsum([face_dof])

# assemble the right-hand side
rhs = bc_val # pressure boundary
rhs[dofs[0]:] += cell_mass * source
    
perm0 = 1
beta0 = 10
gravity = 9.81

q_old = np.zeros(face_dof)
tol = 1e-4
err_rel = tol + 1
it_num = 0
while err_rel > tol:
    
    cell_q_old = (face_proj * q_old).reshape((3, -1), order="F")  
    for sd, data in mdg.subdomains(return_data=True):
        
        norm_q_old =  np.linalg.norm(cell_q_old, axis=0)
        ## Compute F_0 for each cell, and check if F_0 > 0.1
        #F_0 = beta0 * perm0 * norm_q_old / gravity
        #E = F_0 / (1+F_0)
                
        #beta = np.zeros(sd.num_cells)
        #beta[sd.cell_centers[1, :] > 0.5] = 1
        #beta[E > 0.11] = beta0
        
        effective_perm = 1/(1/perm0 + beta0 * norm_q_old)
        data[pp.PARAMETERS][keyword]["second_order_tensor"] = pp.SecondOrderTensor(effective_perm)

    # construct the local matrices
    face_mass = pg.face_mass(mdg, discr=pp.MVEM(keyword))

    # assemble the saddle point problem
    spp = sps.bmat([[face_mass, -div.T], 
                    [      div,   None]], format="csc")

    # solve the problem
    ls = pg.LinearSystem(spp, rhs)
    ls.flag_ess_bc(np.hstack(bc_ess), np.hstack(bc_val_ess)) # the bc_val_ess are flux-boundary
    x = ls.solve()

    # extract the variables
    q, p = np.split(x, dofs)
    
    err_abs = np.linalg.norm(q-q_old)
    norm_q_old = np.linalg.norm(q_old)
    err_rel = err_abs / norm_q_old if norm_q_old else err_abs
    
    q_old = q
    it_num += 1
    print(it_num, "step", err_rel)

1 step 1.0246950765959597
2 step 0.9090909090909094
3 step 4.761904761904758
4 step 0.6939625260235947
5 step 1.3964530093562353
6 step 0.46236146494606584
7 step 0.5794410827204303
8 step 0.2807906536921545
9 step 0.2737415790965305
10 step 0.1610280867570529
11 step 0.13720954340022856
12 step 0.08932590221640553
13 step 0.07081503246921884
14 step 0.04863870130331691
15 step 0.037099970418509626
16 step 0.026216402235876964
17 step 0.019589281325624843
18 step 0.014053347544831587
19 step 0.010386128861393987
20 step 0.0075111542007385386
21 step 0.00551870498567554
22 step 0.004008197177111279
23 step 0.0029357850757763338
24 step 0.0021371060635250296
25 step 0.0015627128014255103
26 step 0.0011389593152319626
27 step 0.0008321021169970741
28 step 0.000606857163115612
29 step 0.0004431492104888914
30 step 0.0003233027593123619
31 step 0.00023602812196322896
32 step 0.0001722276417894533
33 step 0.00012571845883125308
34 step 9.174461695238224e-05


Since the computed $q$ is one value per facet of the grid, for visualization purposes we project the flux in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [10]:
# post process velocity
face_proj = pg.proj_faces_to_cells(mdg, discr=pp.MVEM(keyword))
cell_q = (face_proj * q).reshape((3, -1), order="F")

for _, data in mdg.subdomains(return_data=True):
    data[pp.STATE] = {"cell_q": cell_q, "p": p}

save = pp.Exporter(mdg, "sol")
save.write_vtu(["cell_q", "p"])

A representation of the computed solution is given below, where the cells are colored with $p$ and the arrows are the $q$. <br>
![](fig/darcy.png)